<a href="https://colab.research.google.com/github/yash-02/Live-Face-Mask-Detection-Deep-Learning/blob/master/Face_Mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting data from kaggle

In [1]:
! mkdir ~/.kaggle

In [2]:
! cp kaggle.json ~/.kaggle/ 

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
! kaggle datasets list

ref                                                       title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ruchi798/data-science-job-salaries                        Data Science Job Salaries                            7KB  2022-06-15 08:59:12           6226        208  1.0              
surajjha101/bigbasket-entire-product-list-28k-datapoints  BigBasket Entire Product List (~28K datapoints)      6MB  2022-06-22 12:51:18           1978         88  1.0              
victorsoeiro/netflix-tv-shows-and-movies                  Netflix TV Shows and Movies                          2MB  2022-05-15 00:01:23          16642        483  1.0              
sameepvani/nasa-nearest-earth-objects                     NASA - Nearest Earth Objects         

In [ ]:
! kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

 94% 311M/330M [00:01<00:00, 188MB/s]
100% 330M/330M [00:01<00:00, 177MB/s]


In [ ]:
!unzip face-mask-12k-images-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Face Mask Dataset/Train/WithoutMask/1855.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1856.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1857.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1858.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1859.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1860.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1861.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1862.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1863.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1864.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1865.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1866.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1867.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1868.png  
  inflating: Face Mask Dataset/Train/WithoutMask/1869.png  
  inflating: Face Mask Dataset/Train/WithoutMask/

#Import

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import pandas
import numpy as np
import os

In [ ]:
img = cv2.imread('/content/Face Mask Dataset/Train/WithMask/10.png')
img.shape

(82, 82, 3)

## Data Creation

### Data Argumentation

In [ ]:
TRAINING_DIR = "/content/Face Mask Dataset/Train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                  #  shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')


Generating Traing and validation data

In [ ]:
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=32, 
                                                    target_size=(80,80),
                                                    class_mode="binary",
                                                    shuffle = True,
                                                    seed = 41
                                                    )
VALIDATION_DIR = "/content/Face Mask Dataset/Test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=32, 
                                                         class_mode = 'binary',
                                                         target_size=(80,80),
                                                        shuffle = True,
                                                        seed = 41)

Found 10000 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


## Model Building 

In [ ]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(80, 80, 3)),
    Conv2D(100, (3,3), activation='relu'),

    MaxPooling2D(2,2),

    Conv2D(100, (3,3), activation='relu'),
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')

])
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model', 
                             monitor='val_loss', 
                             verbose=0,
                             save_best_only=True,mode='auto')

In [ ]:
history = model.fit(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10
313/313 [==============================] - 48s 116ms/step - loss: 0.2915 - accuracy: 0.8790 - val_loss: 0.1646 - val_accuracy: 0.9405
Epoch 2/10
313/313 [==============================] - 35s 112ms/step - loss: 0.2070 - accuracy: 0.9214 - val_loss: 0.1400 - val_accuracy: 0.9516
Epoch 3/10
313/313 [==============================] - 34s 109ms/step - loss: 0.2045 - accuracy: 0.9259 - val_loss: 0.1493 - val_accuracy: 0.9425
Epoch 4/10
313/313 [==============================] - 34s 110ms/step - loss: 0.1795 - accuracy: 0.9340 - val_loss: 0.1369 - val_accuracy: 0.9567
Epoch 5/10
313/313 [==============================] - 34s 110ms/step - loss: 0.1856 - accuracy: 0.9334 - val_loss: 0.1127 - val_accuracy: 0.9698
Epoch 6/10
313/313 [==============================] - 34s 108ms/step - loss: 0.1626 - accuracy: 0.9415 - val_loss: 0.1150 - val_accuracy: 0.9546
Epoch 7/10
313/313 [==============================] - 34s 108ms/step - loss: 0.1691 - accuracy: 0.9388 - val_loss: 0.1416 - val_ac

In [ ]:
pd.DataFrame(history.history).plot(fig)

In [ ]:
model.save('Face_mask_model_2')

INFO:tensorflow:Assets written to: Face_mask_model_2/assets


In [ ]:
! unzip "/content/face_mask_model_.zip"

Archive:  /content/face_mask_model_.zip
   creating: face_mask_model_2/assets/
  inflating: face_mask_model_2/keras_metadata.pb  
  inflating: face_mask_model_2/saved_model.pb  
   creating: face_mask_model_2/variables/
  inflating: face_mask_model_2/variables/variables.data-00000-of-00001  
  inflating: face_mask_model_2/variables/variables.index  


In [ ]:
model = load_model('/content/face_mask_model_2')
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 78, 78, 100)       2800      
                                                                 
 conv2d_17 (Conv2D)          (None, 76, 76, 100)       90100     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 38, 38, 100)      0         
 2D)                                                             
                                                                 
 conv2d_18 (Conv2D)          (None, 36, 36, 100)       90100     
                                                                 
 conv2d_19 (Conv2D)          (None, 34, 34, 100)       90100     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 17, 17, 100)      0         
 2D)                                                  

In [ ]:
img = cv2.imread('/content/Face Mask Dataset/Train/WithoutMask/1005.png')/255.
img = stretch_near = cv2.resize(img, (80,80),
               interpolation = cv2.INTER_NEAREST)
print(img.shape)
img = np.reshape(img,(1,80,80,3))

img.shape
model.predict(img)

(80, 80, 3)


array([[0.97397316, 0.02602684]], dtype=float32)

In [ ]:
np.argmax([0.97397316, 0.02602684])

0

In [ ]:
classes = {0:"Mask",
           1:"No Mask"}

In [ ]:
classes[1]

'No Mask'

In [ ]:
def prediction(model,data):
  img = cv2.imread(data,)/255.
  img = stretch_near = cv2.resize(img, (80,80),
               interpolation = cv2.INTER_NEAREST)
  # print(img.shape)
  img = np.reshape(img,(1,80,80,3))
  # img.shape
  return  model.predict(img)

In [ ]:

import random as rd
import matplotlib.pyplot as plt


In [ ]:
def plotpred():
  test_list1 = ["/content/Face Mask Dataset/Train/WithMask/"+i for i in rd.sample(os.listdir('/content/Face Mask Dataset/Train/WithMask'),5)]
  test_list2 = ["/content/Face Mask Dataset/Train/WithoutMask/"+i for i in rd.sample(os.listdir('/content/Face Mask Dataset/Train/WithoutMask'),5)]
  c=0
  plt.figure(figsize=(16,6))
  for path in (test_list1+test_list2):
    c+=1
    # print(path)
    img = cv2.imread(path)
    plt.subplot(2,5,c)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    print(np.argmax(prediction(model,path)))
    plt.title(f"Prediction:{classes[np.argmax(prediction(model,path))]}")
    plt.subplots_adjust(wspace= 0.2,hspace = 0.2)
    plt.imshow(img)
    plt.axis(False)
plotpred()

In [ ]:
for import shutil
shutil.make_archive('face_mask_model_2', 'zip', '/content/Face_mask_model_2')


'/content/face_mask_model_2.zip'